In [9]:
pip install lightgbm

You are using pip version 18.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [17]:
import scipy.sparse
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV 
from lightgbm import LGBMRegressor
from scipy.stats import randint as sp_randint
from scipy.stats import uniform


In [2]:
X_train = scipy.sparse.load_npz("train_final.npz")
y_train = np.load('y_train.npy')

X_cv = scipy.sparse.load_npz("cv_final.npz")
y_cv = np.load('y_cv.npy')

X_test = scipy.sparse.load_npz("test_final.npz")
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((881245, 755184), (881245,), (155514, 755184), (155514,))

In [12]:
from sklearn.feature_selection.univariate_selection import SelectKBest, f_regression

fselect = SelectKBest(f_regression, k=50000)
train_features = fselect.fit_transform(X_train, y_train)
cv_features = fselect.transform(X_cv)
test_features = fselect.transform(X_test)

In [19]:
train_features.shape


(881245, 70000)

In [20]:
X_train.shape, y_train.shape

((881245, 755184), (881245,))

In [21]:
lgb_model = LGBMRegressor(subsample=0.9)

params = {'learning_rate': uniform(0, 1),
          'n_estimators': sp_randint(200, 1500),
          'num_leaves': sp_randint(20, 200),
          'max_depth': sp_randint(2, 15),
          'min_child_weight': uniform(0, 2),
          'colsample_bytree': uniform(0, 1),
         }
lgb_random = RandomizedSearchCV(lgb_model, param_distributions=params, n_iter=10, cv=3, random_state=56, 
                                scoring='neg_root_mean_squared_error', verbose=10, return_train_score=True)
lgb_random = lgb_random.fit(train_features, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] colsample_bytree=0.9841918505425767, learning_rate=0.33341226601465335, max_depth=4, min_child_weight=0.8260007976947785, n_estimators=854, num_leaves=107 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [22]:
best_params = {'colsample_bytree': 0.44583275285359114, 'learning_rate': 0.09997491581800289, 'max_depth': 12, 'min_child_weight': 1.7323522915498704, 'n_estimators': 1323, 'num_leaves': 123}

In [23]:
model = LGBMRegressor(**best_params, subsample=0.9, random_state=56, n_jobs=-1)
model.fit(train_features, y_train)

KeyboardInterrupt: 

In [27]:
first = pd.read_csv('./lgbmTry.csv')

In [26]:
second = pd.read_csv('./ridge_submission_3.5.csv')

In [28]:
submission_df = pd.DataFrame(first['id'])
print(submission_df.shape)
submission_df.head()

(444761, 1)


,id
0,1048243
1,86986
2,698316
3,268868
4,1360398


In [29]:
submission_df['price'] = first['price']*0.5 + second['price']*0.5 

submission_df.to_csv('combined.csv', index=False)